# DataGrid

In [ ]:
from ipywxyz import DataGrid, AlphaColorPicker, TextRenderer
from ipywxyz.widgets.widget_datagrid import FixedFunc
import pandas as pd, traitlets as T, ipywidgets as W

In [ ]:
def make_grid():
    dg = DataGrid(layout=dict(height="400px", flex="1"))

    with W.Output():
        @W.interact
        def data(rows=(1, 1000), cols=(1, 1000)):
            dg.value = pd.DataFrame((pd.np.arange(rows * cols) / 100.0).reshape((rows, cols)))
    x = W.IntSlider(description='scroll x')
    y = W.IntSlider(description='scroll y', orientation='vertical')
    rs = W.IntSlider(description='row height', orientation='vertical')
    cs = W.IntSlider(description='column width')
    hr = W.HTML()
    hc = W.HTML()

    T.link((x, "value"), (dg, "scroll_x"))
    T.dlink((dg, "max_x"), (x, "max"))
    T.link((y, "value"), (dg, "scroll_y"))
    T.dlink((dg, "max_y"), (y, "max"))
    T.link((dg, "row_size"), (rs, "value"))
    T.link((dg, "column_size"), (cs, "value"))
    T.dlink((dg, "hover_row"), (hr, "value"), "row {}".format)
    T.dlink((dg, "hover_column"), (hc, "value"), "column {}".format)
    colors = [ 
        [p, T.link((dg, t), (p, "value"))][0]
        for t in dg.traits().keys()
        for p in [AlphaColorPicker(description=t.replace('_color', '').replace('_', ' '))]
        if t.endswith('_color')
    ]
    grid_playground = W.HBox([
        W.VBox([
            W.HBox([W.VBox([hr, hc]), x, cs, data.widget]), 
            dg, 
        ], layout=dict(flex="1")),
        W.VBox([y, rs, *colors]), 
    ])
    grid_playground.dg = dg
    return grid_playground

In [ ]:
grid_playground = make_grid()
grid_playground

Cell renderers are still a work-in-progress.

In [ ]:
grid_renderer_playground = make_grid()
btn = W.Button(description="Custom Renderers")
def _btn_click(*_):
    grid_renderer_playground.dg.cell_renderers = []

    dg = grid_renderer_playground.dg
    with dg.hold_trait_notifications():
        dg.grid_line_color = dg.background_color = dg.header_background_color = dg.header_grid_line_color = 'rgba(0, 0, 0, 0)'
        dg.void_color = "black"
        dg.row_size = 40

        grid_renderer_playground.dg.cell_renderers = [
            TextRenderer(text_color="#ccc", font="14px sans-serif", format_func=FixedFunc(digits=1)),
            TextRenderer(text_color="#fff", font="bold 14px sans-serif", region="row-header"),
            TextRenderer(text_color="#2196f3", font="bold 24px sans-serif", region="column-header")
        ]
btn.on_click(_btn_click)
grid_renderer_playground.children = (btn,) + grid_renderer_playground.children
grid_renderer_playground